In [50]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
df=sns.load_dataset('tips')

In [52]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [54]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [55]:
y= df['total_bill']
x = df.drop('total_bill',axis=1)

In [56]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3)

## encoding

In [57]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [58]:
from sklearn.preprocessing import LabelEncoder
le1= LabelEncoder()
le2= LabelEncoder()
le3= LabelEncoder()

x_train['sex']=le1.fit_transform(x_train['sex'])
x_train['smoker']  =le2.fit_transform(x_train['smoker'])
x_train['time']=le3.fit_transform(x_train['time'])


In [59]:
x_train.head()

,tip,sex,smoker,day,time,size
26,2.00,1,0,Sat,0,2
3,3.31,1,0,Sun,0,2
28,4.30,1,0,Sat,0,2
112,4.00,1,0,Sun,0,3
140,3.50,0,0,Thur,1,2


In [60]:
x_test['sex']=le1.transform(x_test['sex'])
x_test['smoker']  =le2.transform(x_test['smoker'])
x_test['time']=le3.transform(x_test['time'])

In [61]:
# one hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('one hot',OneHotEncoder(drop='first'),[3])],remainder='passthrough')


In [62]:
import sys
np.set_printoptions(threshold=sys.maxsize)
x_train = ct.fit_transform(x_train)

In [63]:
x_test= ct.transform(x_test)

In [68]:
from sklearn.svm import SVR
model = SVR()
model.fit(x_train,y_train)


SVR()

In [72]:
ypred = model.predict(x_test)
from sklearn.metrics import r2_score,mean_absolute_error
print(r2_score(y_test,ypred))
print(mean_absolute_error(y_test,ypred))

0.46571784359692503
4.28435187508517


# hyper parameter tunning


In [73]:
from sklearn.model_selection import GridSearchCV
pramete={
    'C':[1,100,0.1,0.01],
    'gamma':[1,0.1,0.01,0.001,0.0001],
    'kernel':['rbf','sigmoid']
}
grid = GridSearchCV(SVR(),param_grid=pramete,cv=5,refit=True,verbose=3)

grid.fit(x_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END .........C=1, gamma=1, kernel=rbf;, score=-0.007 total time=   0.0s
[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.054 total time=   0.0s
[CV 3/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.148 total time=   0.0s
[CV 4/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.166 total time=   0.0s
[CV 5/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.125 total time=   0.0s
[CV 1/5] END .....C=1, gamma=1, kernel=sigmoid;, score=-0.244 total time=   0.0s
[CV 2/5] END .....C=1, gamma=1, kernel=sigmoid;, score=-0.138 total time=   0.0s
[CV 3/5] END .....C=1, gamma=1, kernel=sigmoid;, score=-0.007 total time=   0.0s
[CV 4/5] END .....C=1, gamma=1, kernel=sigmoid;, score=-0.001 total time=   0.0s
[CV 5/5] END .....C=1, gamma=1, kernel=sigmoid;, score=-0.086 total time=   0.0s
[CV 1/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.237 total time=   0.0s
[CV 2/5] END ........C=1, gamma=0.1, kernel=rbf

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 100, 0.1, 0.01],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'sigmoid']},
             verbose=3)

In [74]:
grid.best_params_

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

In [75]:
ypred_grid = grid.predict(x_test)

In [76]:
print(r2_score(y_test,ypred_grid))
print(mean_absolute_error(y_test,ypred_grid))

0.5248413749980602
4.029872987542261
